In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sparse
import lmdb

# Add paths
sys.path.insert(0, os.path.abspath('../../../python/')) # PetaVision
sys.path.insert(0, os.path.abspath(os.environ['HOME']+'/Work/Libraries/caffe/python/')) # Caffe

# Import aux libraries
from pvtools import *
import caffe

In [2]:
#set up path to pvp file
pvp_file_path = os.environ['HOME']+'/Work/LANL/Data/S1_400.pvp'
image_list_path = os.environ['HOME']+'/Work/LANL/Data/mixed_cifar.txt'
lmdb_file_name = 'pvCifar_train_lmdb'
proto_file_name = 'mlp.prototxt'
solver_file_name = 'solver.prototxt'

#MLP params
batch_size = 64       #Batch size for training with SGD
num_ip1_params = 500  #Number of free parameters in ip1
ip1_lrm_weights = 1   #Learning rate multiplier for ip1 weights
ip1_lrm_bias = 2      #Learning rate multiplyer for ip1 bias
ip2_lrm_weights = 1   #Learning rate multiplier for ip2 weights
ip2_lrm_bias = 2      #Learning rate multiplier for ip2 bias

In [3]:
def pvObj2DenseMat(pvObj):
    numIm = len(pvObj)
    numF = pvObj.header['nf']
    numY = pvObj.header['ny']
    numX = pvObj.header['nx']
    dense_shape = numF * numY * numX
    out_mat = np.zeros((numIm, numF, numY, numX))
    for frame_idx in range(numIm):
        frame_indices = np.array(pvObj[frame_idx].values)[:,0].astype(np.int32)
        frame_data = np.array(pvObj[frame_idx].values)[:,1].astype(np.float32)
        numActive = len(frame_indices)
        ij_mat = (np.zeros(numActive), frame_indices)
        out_vec = np.array(sparse.coo_matrix((frame_data, ij_mat), shape=(1, dense_shape)).todense())
        out_mat[frame_idx,:,:,:] = out_vec.reshape((numF, numY, numX))
    return out_mat

In [4]:
def cifarList2Vec(file_loc,label_pos):
    label_list = []
    with open(file_loc, 'r') as f:
        for line in f:
            line_arry = line.split('/')
            label_list.append(line_arry[label_pos])
    return np.array(label_list)

In [5]:
pvData = readpvpfile(pvp_file_path) # This takes some time...
pvActivities = pvObj2DenseMat(pvData)
labels = cifarList2Vec(image_list_path, 6).astype(np.int64)
assert labels.shape[0] == pvActivities.shape[0]

In [6]:
# Database size set to be 10x bigger than needed
# as suggested in http://deepdish.io/2015/04/28/creating-lmdb-in-python/
map_size = pvActivities.nbytes * 10

env = lmdb.open(lmdb_file_name, map_size=map_size)
with env.begin(write=True) as txn:   # txn is a Transaction object
    for i in range(len(pvData)):
        datum = caffe.proto.caffe_pb2.Datum()
        datum.channels = pvActivities.shape[1]
        datum.height = pvActivities.shape[2]
        datum.width = pvActivities.shape[3]
        datum.data = pvActivities[i].tobytes()
        datum.label = int(labels[i])
        str_id = '{:08}'.format(i)
        txn.put(str_id.encode('ascii'), datum.SerializeToString())

In [7]:
from caffe import layers as L, params as P, to_proto

netSpec = caffe.NetSpec()
netSpec.data, netSpec.label = L.Data(batch_size=batch_size, backend=P.Data.LMDB, source=lmdb_file_name, \
        transform_param=dict(scale=1./255), phase=0, ntop=2)


netSpec.ip1 = L.InnerProduct(netSpec.data, num_output=num_ip1_params, \
        weight_filler={'type': 'xavier'}, \
        bias_filler={'type': 'constant', 'value': 0})
netSpec.ip1.fn.params['param'] = [{'lr_mult': ip1_lrm_weights, 'decay_mult': 1}, \
                                  {'lr_mult': ip1_lrm_bias, 'decay_mult': 1}]

netSpec.relu1 = L.ReLU(netSpec.ip1, in_place=True)

netSpec.ip2 = L.InnerProduct(netSpec.data, num_output=10, \
        weight_filler={'type': 'xavier'}, \
        bias_filler={'type': 'constant', 'value': 0})
netSpec.ip2.fn.params['param'] = [{'lr_mult': ip2_lrm_weights, 'decay_mult': 1}, \
                                  {'lr_mult': ip2_lrm_bias, 'decay_mult': 1}]

#netSpec.accuracy = L.Accuracy(netSPec.ip2, netSpec.label, include={phase: 'TEST'})

netSpec.loss = L.SoftmaxWithLoss(netSpec.ip2, netSpec.label)

open(proto_file_name,"w").write(str(netSpec.to_proto()))

In [ ]:
if not os.path.exists('checkpoints'):
    os.makedirs('checkpoints')

solver = caffe.SGDSolver(solver_file_name)
net = solver.net
solver.solve()